### Dataset

In this homework, we'll build a model for classifying various hair types. 
For this, we will use the Hair Type dataset that was obtained from 
[Kaggle](https://www.kaggle.com/datasets/kavyasreeb/hair-type-dataset) 
and slightly rebuilt.

You can download the target dataset for this homework from 
[here](https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip):

```bash
wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
unzip data.zip
```
The dataset is split into train and test dataset. Use train dataset to train the model and test dataset for validation. 

In the lectures we saw how to use a pre-trained neural network. In the homework, we'll train a much smaller model from scratch. 

We will use PyTorch for that.

You can use Google Colab or your own computer for that.

### Data Preparation

The dataset contains around 1000 images of hairs in the separate folders 
for training and test sets. 

### Reproducibility

Reproducibility in deep learning is a multifaceted challenge that requires attention 
to both software and hardware details. In some cases, we can't guarantee exactly the same results during the same experiment runs.

Therefore, in this homework we suggest to set the random number seed generators by:

```python
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
```

Also, use PyTorch of version 2.8.0 (that's the one in Colab).

In [2]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Model

For this homework we will use Convolutional Neural Network (CNN). We'll use PyTorch.

You need to develop the model with following structure:

* The shape for input should be `(3, 200, 200)` (channels first format in PyTorch)
* Next, create a convolutional layer (`nn.Conv2d`):
    * Use 32 filters (output channels)
    * Kernel size should be `(3, 3)` (that's the size of the filter), padding = 0, stride = 1
    * Use `'relu'` as activation 
* Reduce the size of the feature map with max pooling (`nn.MaxPool2d`)
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using `flatten` or `view`
* Next, add a `nn.Linear` layer with 64 neurons and `'relu'` activation
* Finally, create the `nn.Linear` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use `torch.optim.SGD` with the following parameters:

* `torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)`

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class HairTypeCNN(nn.Module):
    def __init__(self):
        super(HairTypeCNN, self).__init__()

        # 1) CONVOLUTION LAYER: in_channels=3, out_channels=32, kernel_size=3

        # ---------------------------------------------------------
        #   "Add a convolution layer with 32 filters, kernel size 3,
        #    stride 1, padding 0"
        #
        # In TensorFlow:
        #   tf.keras.layers.Conv2D(32, 3, strides=1, padding="valid")
        #
        # In PyTorch:
        #   nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        #
        # Input shape:  (3, 200, 200)
        # Output shape: (32, 198, 198)
        # ---------------------------------------------------------

        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=32,
            kernel_size=3,
            stride=1,
            padding=0
        )

        # 2) MAX-POOL LAYER 2x2
        # In TensorFlow:
        #   tf.keras.layers.MaxPooling2D(pool_size=2)
        #
        # In PyTorch:
        #   nn.MaxPool2d(kernel_size, stride)
        #
        # Conv output: (32, 198, 198)
        # After pooling: (32, 99, 99)
        # ------------------------------

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # After conv + pool:
        # Input:  (3, 200, 200)
        # Conv:   (32, 198, 198)
        # Pool:   (32, 99, 99)

        # 3) COMPUTE FLATTEN SIZE
        # After Conv → Pool we know the tensor is:
        #   (batch, 32, 99, 99)
        # Flatten size = 32 * 99 * 99 = 313632
        self.flatten_size = 32 * 99 * 99

        # 4) FIRST FULLY CONNECTED LAYER (dense layer)
        #   "Add a fully-connected layer with 64 units"
        #
        # In TensorFlow:
        #   tf.keras.layers.Dense(64, activation="relu")
        #
        # PyTorch separates Linear + activation
        # ----------------------------------------
        self.fc1 = nn.Linear(self.flatten_size, 64)

        # 5) OUTPUT LAYER
        #   "Add the output layer with 1 neuron"
        #
        # Binary classification → output logits
        # (Note: use BCEWithLogitsLoss later)
        #
        # In TensorFlow:
        #   Dense(1, activation="sigmoid")
        #
        # In PyTorch:
        #   Linear() gives raw logits
        #   Sigmoid is applied inside the loss function
        self.fc2 = nn.Linear(64, 1)  # 1 output neuron for binary classification

    def forward(self, x):
        # INPUT SHAPE: (batch, 3, 200, 200)
        # x: (batch_size, 3, 200, 200)

        # 1) Convolution
        x = self.conv1(x)

        # Activation (PyTorch requires explicit activation)
        x = F.relu(x)

        # 2) Max Pooling
        x = self.pool(x)

        # 3) Flatten (TensorFlow does this automatically)
        x = x.view(x.size(0), -1)  # (batch_size, 313632)

        # 4) Dense layer + activation
        # Fully connected
        x = self.fc1(x)
        x = F.relu(x)

        # 5) Output layer
        x = self.fc2(x)  # logits, no sigmoid here

        return x


In [5]:
# instantiate the model and move it to the appropriate device
model = HairTypeCNN().to(device)


### Question 1

Which loss function you will use?

> `nn.BCEWithLogitsLoss()`

**ANSWER 👩🏽‍💻**

- I am doing binary classification with a single output neuron.
- My model returns raw logits (no sigmoid inside the model).
- `nn.BCEWithLogitsLoss()` is the correct loss for this setup because:
    - It expects one output per sample (logit), exactly like my model produces.
    - It internally applies a sigmoid + binary cross-entropy in a numerically stable way.
    - It is the PyTorch equivalent of using `Dense(1, activation="sigmoid")` + `BinaryCrossentropy` in TensorFlow.

The other options don't match the task:

- MSELoss treats the problem as regression → not ideal.
- CrossEntropyLoss requires two output classes, not a single logit.
- CosineEmbeddingLoss is for similarity learning, not classification.

So, `nn.BCEWithLogitsLoss()` is the correct and most stable loss for binary classification with a single-logit output.

### Question 2

What's the total number of parameters of the model? You can use `torchsummary` or count manually. 

In PyTorch, you can find the total number of parameters using:

```python
# Option 1: Using torchsummary (install with: pip install torchsummary)
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

# Option 2: Manual counting
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")
```

> 20073473

In [7]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 20073473


### Generators and Training

For the next two questions, use the following transformation for both train and test sets.

* We don't need to do any additional pre-processing for the images.
* Use `batch_size=20`
* Use `shuffle=True` for both training, but `False` for test. 


In [10]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [24]:
validation_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [26]:
# datasets

train_dataset = datasets.ImageFolder(
    root='data/train',
    transform=train_transforms
)

validation_dataset = datasets.ImageFolder(
    root='data/test',
    transform=validation_transforms
)


In [27]:
# data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True
)

validation_loader = DataLoader(
    validation_dataset,
    batch_size=32,
    shuffle=False
)


Now fit the model.

You can use this code:

In [ ]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
criterion = nn.BCEWithLogitsLoss()


for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")